In [6]:
!pip install yt-dlp

In [7]:
# Set AssemblyAI API Key
import os
from pathlib import Path

from dotenv import load_dotenv
# ❷ Read the key-value pairs and add them to os.environ
load_dotenv()      # ← now the vars are set

# print(f"AssemblyAI API key is {os.environ['ASSEMBLYAI_API_KEY']} !")

True

In [8]:
import os
import time
import requests

# ─── AssemblyAI Configuration ─────────────────────────────────────────────────

API_KEY = os.getenv("ASSEMBLYAI_API_KEY")
if not API_KEY:
    raise RuntimeError("Please set your ASSEMBLYAI_API_KEY environment variable")

# Fixed headers - use proper Authorization header format
HEADERS = {
    "Authorization": API_KEY,  # Changed from "authorization" to "Authorization"
    "Content-Type": "application/json"  # Changed from "content-type" to "Content-Type"
}
TRANSCRIBE_URL = "https://api.assemblyai.com/v2/transcript"

# ─── Transcription Function ────────────────────────────────────────────────────

def transcribe_url(
    audio_url: str,
    poll_interval: int = 5,
    timeout: int = 600
) -> str:
    """
    Submit an audio file URL to AssemblyAI for transcription and poll until completion.
    
    Args:
        audio_url: Direct URL to the audio file (MP3, WAV, etc.)
        poll_interval: Seconds between polling requests (default: 5)
        timeout: Maximum seconds to wait before giving up (default: 600)
    
    Returns:
        str: The transcribed text
        
    Raises:
        requests.HTTPError: For API request issues
        RuntimeError: For transcription errors
        TimeoutError: If transcription takes too long
    """
    print(f"Starting transcription for: {audio_url}")
    
    # Submit transcription request
    response = requests.post(
        TRANSCRIBE_URL,
        json={"audio_url": audio_url},
        headers=HEADERS
    )
    response.raise_for_status()
    transcript_id = response.json()["id"]
    print(f"Transcription job started with ID: {transcript_id}")

    # Poll for completion
    start_time = time.time()
    while True:
        poll_resp = requests.get(f"{TRANSCRIBE_URL}/{transcript_id}", headers=HEADERS)
        poll_resp.raise_for_status()
        data = poll_resp.json()
        status = data.get("status")

        print(f"Status: {status}")
        
        if status == "completed":
            print("Transcription completed!")
            return data.get("text", "")
        elif status == "error":
            error_msg = data.get("error", "Unknown error")
            raise RuntimeError(f"Transcription failed: {error_msg}")

        if time.time() - start_time > timeout:
            raise TimeoutError(f"Transcription timed out after {timeout} seconds")

        time.sleep(poll_interval)

# ─── Usage Example ─────────────────────────────────────────────────────────────

# Example usage:
# audio_url = "https://example.com/podcast.mp3"
# transcript = transcribe_url(audio_url)
# print("Transcript:", transcript)

In [9]:
# ─── YouTube Transcription Setup ──────────────────────────────────────────────

# Replace with your actual YouTube URL
youtube_url = "https://www.youtube.com/watch?v=Q1QPXyebhiY"

# Required imports for YouTube transcription
import yt_dlp
import tempfile
import os
import requests

def transcribe_youtube_video(
    youtube_url: str,
    poll_interval: int = 5,
    timeout: int = 600
) -> str:
    """
    Download and transcribe YouTube audio using AssemblyAI.
    
    Args:
        youtube_url: YouTube video URL
        poll_interval: seconds between polling
        timeout: max seconds to wait
    Returns:
        The transcript text.
    """
    print(f"Processing YouTube video: {youtube_url}")
    
    # Create a temporary directory to store the download
    with tempfile.TemporaryDirectory() as tmpdir:
        download_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(tmpdir, 'audio'), # Use a fixed name inside the temp dir
            'noplaylist': True,
        }
        
        audio_file = None
        with yt_dlp.YoutubeDL(download_opts) as ydl:
            # Download the file
            ydl.download([youtube_url])
            
            # Find the downloaded file (yt-dlp adds the extension)
            for entry in os.listdir(tmpdir):
                if entry.startswith('audio'):
                    audio_file = os.path.join(tmpdir, entry)
                    break
        
        if not audio_file or not os.path.exists(audio_file) or os.path.getsize(audio_file) == 0:
            raise Exception(f"Failed to download audio file from YouTube.")
    
        print(f"Downloaded to {audio_file} ({os.path.getsize(audio_file)} bytes)")
        
        # Upload to AssemblyAI
        print("Uploading audio to AssemblyAI...")
        try:
            upload_url = upload_file_to_assemblyai(audio_file)
            print(f"Uploaded URL: {upload_url}")
            
            # Transcription is now handled by the caller, just return the URL
            return transcribe_url(upload_url, poll_interval, timeout)
            
        except Exception as e:
            raise e

def upload_file_to_assemblyai(file_path: str) -> str:
    """
    Uploads a local audio file to AssemblyAI and returns a public URL for transcription.
    """
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        raise ValueError(f"File is empty or does not exist: {file_path}")
    
    print(f"Uploading file: {file_path} (size: {os.path.getsize(file_path)} bytes)")
    
    with open(file_path, 'rb') as f:
        resp = requests.post(
            'https://api.assemblyai.com/v2/upload',
            headers={'Authorization': API_KEY},
            data=f
        )
    
    resp.raise_for_status()
    return resp.json().get('upload_url')

In [10]:
# Transcribe a YouTube video
print("Starting YouTube video transcription...")
transcript = transcribe_youtube_video(youtube_url)
print("\n" + "="*50)
print("TRANSCRIPT:")
print("="*50)
print(transcript)

Starting YouTube video transcription...
Processing YouTube video: https://www.youtube.com/watch?v=Q1QPXyebhiY
[youtube] Extracting URL: https://www.youtube.com/watch?v=Q1QPXyebhiY
[youtube] Q1QPXyebhiY: Downloading webpage
[youtube] Q1QPXyebhiY: Downloading webpage
[youtube] Q1QPXyebhiY: Downloading tv client config
[youtube] Q1QPXyebhiY: Downloading tv client config
[youtube] Q1QPXyebhiY: Downloading tv player API JSON
[youtube] Q1QPXyebhiY: Downloading tv player API JSON
[youtube] Q1QPXyebhiY: Downloading ios player API JSON
[youtube] Q1QPXyebhiY: Downloading ios player API JSON
[youtube] Q1QPXyebhiY: Downloading m3u8 information
[youtube] Q1QPXyebhiY: Downloading m3u8 information
[info] Q1QPXyebhiY: Downloading 1 format(s): 251
[info] Q1QPXyebhiY: Downloading 1 format(s): 251
[download] Destination: /var/folders/1p/vtrslkcd30vfb7ndv5b9c5s40000gp/T/tmpl28sghv_/audio
[download]   0.2% of   17.11MiB at    2.48MiB/s ETA 00:07[download] Destination: /var/folders/1p/vtrslkcd30vfb7ndv5b9c5